# Chapitre 4 - API

### 4.3.4 Utilisation d’API sans authentification

In [ ]:
# On importe la librairie
import requests

# On envoie une requête get
response = requests.get('https://api.exemple.com/data')

On peut améliorer le code précédent en affichant la réponse si la requête a correctement abouti.

In [ ]:
if response.status_code == 200:
    data = response.json()
else:
    print(f'Échec de la récupération des données : {response.status_code}')

Pour avoir une idée plus précise du type d'erreur (si elle provient du côté serveur ou du côté client), on peut utiliser la méthode `raise_for_status()`. 

In [ ]:
try:
    response = requests.get('https://api.exemple.com/data')
    response.raise_for_status()
    data = response.json()
except requests.exceptions.HTTPError as http_err:
    print(f'Une erreur HTTP est survenue : {http_err}')
except Exception as err:
    print(f'Une erreur est survenue : {err}')

Si on effectue plusieurs requête, nous utiliserons l'objet `Session()`. Cela permet de réutiliser la connexion TCP sous-jacente.

In [ ]:
with requests.Session() as session:
	try:
		response = session.get('https://api.exemple.com/data')
    	response.raise_for_status()
        data = response.json()
	except requests.exceptions.HTTPError as http_err:
    	print(f'Une erreur HTTP est survenue : {http_err}')
	except Exception as err:
    	print(f'Une erreur est survenue : {err}')

### 4.3.5 Appels API concurrents avec le multithreading

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def recuperer_url(url):
    with requests.Session() as session:
        return session.get(url)

urls = ['https://api.exemple.com/data1', 'https://api.exemple.com/data2']

with ThreadPoolExecutor(max_workers=5) as executor:
    futur_url = {executor.submit(recuperer_url, url): url for url in urls}
    for futur in concurrent.futures.as_completed(futur_url):
        url = futur_url[futur]
        try:
            data = futur.result().json()
        except Exception as exc:
            print(f'{url} a généré une exception : {exc}')
        else:
            print(f'La page {url} fait {len(data)} octets')

### 4.3.6 Requêtes API avec authentification 

**Authentification par Token (Bearer Token)**

In [ ]:
import requests

# Votre token Bearer (habituellement obtenu après un processus d'autorisation)
bearer_token = 'votre_token_bearer'

# Préparation des en-têtes HTTP avec le token d'authentification
headers = {
    'Authorization': f'Bearer {bearer_token}'
}

# Envoi d'une requête GET avec authentification
response = requests.get('https://api.exemple.com/protected', headers=headers)

# Validation de la réponse
if response.ok:
    print('Réponse reçue:', response.json())
else:
    print('Erreur reçue:', response.status_code)

**Authentification de base HTTP (Basic Auth)**

In [ ]:
from requests.auth import HTTPBasicAuth
import requests

response = requests.get(
    'https://api.exemple.com/basic-auth',
    auth=HTTPBasicAuth('utilisateur', 'mot_de_passe')
)

# Assurez-vous que la réponse est valide
if response.ok:
    data = response.json()
else:
    print('Échec de la récupération des données:', response.status_code)

**Authentification via Proxy**

In [ ]:
# On enregistre dans les URLs proxy dans des variables 
# d'environnement
import os

os.environ['HTTP_PROXY']='http://monproxy.fr:1234'
os.environ['HTTPS_PROXY']= 'https://monproxy.fr:1234'

In [ ]:
import requests
import os

# On crée le dictionnaire contenant les URLs proxy
proxies = {
	'http': os.environ['HTTP_PROXY'],
	'https': os.environ['HTTPS_PROXY'] }

r = requests.get('http://www.url.com', proxies=proxies)


**Authentification avec les Cookies**

In [ ]:
mport requests

# Créez une session pour conserver les cookies
session = requests.Session()

# Se connecter pour obtenir le cookie de session
session.post('https://api.exemple.com/login', data={'utilisateur': 'nom', 'motdepasse': 'mdp'})

# Le cookie de session est maintenant stocké dans la session
response = session.get('https://api.exemple.com/protected')

# Vérification de la réponse
if response.ok:
    print('Réponse reçue de l\'API protégée.')
else:
    print('Erreur de requête:', response.status_code)

### 5. Cas pratique

Vous venez d’intégrer une équipe Marketing et on vous demande de déterminer les zones géographiques les plus propices à votre prochaine campagne. Vous ouvrez votre base de données et constatez avec effroi que les adresses ne sont pas normées et que certaines communes ou codes postaux sont mal orthographiés. 

Bref, il va falloir remettre de l’ordre dans tout ça. Pour se faire, nous allons utiliser l’[API Adresse](https://adresse.data.gouv.fr/api-doc/adresse) .
L’idée est la suivante, on fournit en entrée une adresse approximative et nous récupérons une adresse propre depuis l’API. 

D’autres données nous serons probablement utile à des fins de data-visualisation, en particulier les coordonnées GPS.

**Dictionnaire des données :**
<blockquote>
Les coordonnées GeoJSON sont exprimées en WGS-84 (EPSG 4326)  
  
Les attributs retournés sont :

* id : identifiant de l’adresse (clef d’interopérabilité)
* type : type de résultat trouvé
    * housenumber : numéro « à la plaque »
    * street : position « à la voie », placé approximativement au centre de celle-ci
    * locality : lieu-dit
    * municipality : numéro « à la commune »
* score : valeur de 0 à 1 indiquant la pertinence du résultat
* housenumber : numéro avec indice de répétition éventuel (bis, ter, A, B)
* street : nom de la voie
* name : numéro éventuel et nom de voie ou lieu dit
* postcode : code postal
* citycode : code INSEE de la commune
* city : nom de la commune
* district : nom de l’arrondissement (Paris/Lyon/Marseille)
* oldcitycode : code INSEE de la commune ancienne (le cas échéant)
* oldcity : nom de la commune ancienne (le cas échéant)
* context : n° de département, nom de département et de région
* label : libellé complet de l’adresse
* x : coordonnées géographique en projection légale
* y : coordonnées géographique en projection légale
* importance : indicateur d’importance (champ technique)
</blockquote>

In [7]:
import requests
import json

# L'adresse dont nous disposons est la suivante : 4 bd du port
# Nous avons le code postal 43330 et il semble valide, lançons un 
# premier test pour vérifier si nous pouvons extraire une adresse valide
url = "https://api-adresse.data.gouv.fr/search/?q=8+bd+du+port&postcode=44380"

with requests.Session() as session:
    try:
        response = session.get(url)
        response.raise_for_status()
        # On ne sélectionne que le premier item du fichier 
        data = response.json()['features'][0]
        # On utilise la méthode dumps de la librairie json pour 
        # avoir une mise en forme plus esthétique avec print
        print(json.dumps(data, indent=2))
    except requests.exceptions.HTTPError as http_err:
        print(f'Une erreur HTTP est survenue : {http_err}')
    except Exception as err:
        print(f'Une erreur est survenue : {err}')

{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -2.340983,
      47.258811
    ]
  },
  "properties": {
    "label": "8 Boulevard du Port 44380 Pornichet",
    "score": 0.47710291866028703,
    "housenumber": "8",
    "id": "44132_0141_00008",
    "name": "8 Boulevard du Port",
    "postcode": "44380",
    "citycode": "44132",
    "x": 296409.78,
    "y": 6697932.63,
    "city": "Pornichet",
    "context": "44, Loire-Atlantique, Pays de la Loire",
    "type": "housenumber",
    "importance": 0.51129,
    "street": "Boulevard du Port"
  }
}
